## Importando dados sem anomalia

In [ ]:
!gdown --fuzzy https://drive.google.com/file/d/1UgZ1TpYi_-ciUdwsfc9JM6ZN3GsZthty/view?usp=sharing

Downloading...
From: https://drive.google.com/uc?id=1UgZ1TpYi_-ciUdwsfc9JM6ZN3GsZthty
To: /content/dados_sem_anomalias.csv
100% 51.2M/51.2M [00:00<00:00, 179MB/s]


In [ ]:
import pandas as pd

df = pd.read_csv('/content/dados_sem_anomalias.csv')

df.head()

,dispositivo_1,dispositivo_2,dispositivo_3,dispositivo_4,dispositivo_5,dispositivo_6,dispositivo_7,dispositivo_8,dispositivo_9,dispositivo_10,...,dispositivo_42,dispositivo_43,dispositivo_44,dispositivo_45,dispositivo_46,dispositivo_47,dispositivo_48,dispositivo_49,dispositivo_50,falha
0,73.18,61.70,44.79,34.70,64.35,31.37,71.95,46.84,45.40,57.63,...,57.50,49.11,35.51,49.83,35.35,56.37,56.21,50.41,42.17,0
1,48.70,36.58,42.64,51.02,66.17,43.68,51.84,57.06,40.92,33.10,...,42.58,45.03,55.41,56.54,34.13,50.11,49.88,49.82,69.11,0
2,45.65,69.17,48.58,34.39,42.41,41.61,59.15,55.03,59.03,59.72,...,74.03,48.05,39.78,58.47,63.05,54.80,68.53,45.07,71.07,0
3,63.11,49.81,38.17,59.98,61.59,59.39,48.50,55.62,52.20,30.47,...,43.08,47.89,32.30,66.46,54.78,60.01,21.40,53.12,50.01,0
4,28.41,38.22,43.15,39.12,58.32,71.58,36.61,45.84,35.68,45.38,...,58.20,55.04,36.48,52.88,54.85,66.86,50.58,58.64,53.66,0


## Preparando dados para o treinamento

In [ ]:
X = df.drop('falha', axis=1).values
y = df['falha'].values

X.shape, y.shape

((172477, 50), (172477,))

Comentários da aula:

- não pode usar dados de testes os mesmos dados do treinamento
- em uma matriz de confusão o pior caso é o falso negativo
- Métricas:
  - acurácia (acurácia balanceada)
  - recuperação (recal)
  - precisão (precision)
  - f1 (média da recuperação e da precisão) -> balanceada pelo número de dados


  Como nossos dados estão 50% cada acurária funciona, mas bom usar o F1 por garantia
  

### Separando conjunto em dados de treinamento e de teste

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Importando classificadores

In [ ]:
from sklearn.linear_model import LogisticRegression, PassiveAggressiveClassifier
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier, HistGradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
import time

tecnicas = [
    LogisticRegression(random_state=42),
    PassiveAggressiveClassifier(random_state=42),
    AdaBoostClassifier(random_state=42),
    BaggingClassifier(random_state=42),
    ExtraTreesClassifier(random_state=42),
    GradientBoostingClassifier(random_state=42),
    HistGradientBoostingClassifier(random_state=42),
    RandomForestClassifier(random_state=42),
    KNeighborsClassifier(),
    DecisionTreeClassifier(random_state=42),
]

resultados = []

Comentários Classificadores:
- hiperparametro: parametro de ajuste para treinamento da técnica
- parametro: parametro dentro do modelo treinado

## Realizando treinamento

In [ ]:
for clf in tecnicas:
  nome = clf.__class__.__name__
  print(nome)

  t0 = time.time()
  clf.fit(X_train, y_train)
  tempo = time.time() - t0

  y_pred = clf.predict(X_test)
  acc = accuracy_score(y_test, y_pred)
  f1 = f1_score(y_test, y_pred)

  resultados.append({
      'metodo':nome,
      'acurácia':acc,
      'f1':f1,
      'tempo': tempo,
  })

res = pd.DataFrame(resultados)
res.sort_values('f1',ascending=False)

LogisticRegression
PassiveAggressiveClassifier
AdaBoostClassifier
BaggingClassifier
ExtraTreesClassifier
GradientBoostingClassifier
HistGradientBoostingClassifier
RandomForestClassifier
KNeighborsClassifier
DecisionTreeClassifier


,metodo,acurácia,f1,tempo
7,RandomForestClassifier,0.927644,0.926106,170.592032
6,HistGradientBoostingClassifier,0.918454,0.917228,8.514077
4,ExtraTreesClassifier,0.918715,0.917046,48.703344
3,BaggingClassifier,0.913149,0.910524,128.578601
9,DecisionTreeClassifier,0.857868,0.857896,17.188419
5,GradientBoostingClassifier,0.852968,0.850375,278.168772
8,KNeighborsClassifier,0.761596,0.761582,0.017353
2,AdaBoostClassifier,0.757334,0.758684,49.147989
0,LogisticRegression,0.741912,0.741260,1.378625
1,PassiveAggressiveClassifier,0.542295,0.156886,0.867982


Como pode-se observar teve classificadores que obteram um bom resultado para base mas outros que tiveram muito pouco acertos. Algo relevante de se considerar, que a porcentagem de acertos não está vinculada ao tempo de treinamento, já que ele varia entre os métodos que ficaram melhores elencados.